# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,18.70,56,0,12.86,US,1706663122
1,1,bredasdorp,-34.5322,20.0403,18.64,95,58,2.22,ZA,1706663122
2,2,papatowai,-46.5619,169.4708,18.68,58,100,3.27,NZ,1706663122
3,3,udachny,66.4167,112.4000,-27.15,100,98,1.04,RU,1706663122
4,4,saint-philippe,-21.3585,55.7679,25.77,84,47,3.89,RE,1706662892


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:


# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', 
                                       size='Humidity', color='blue', 
                                       hover_cols=['City', 'Humidity'], 
                                       title='City Map with Humidity')

# Display the map
map_plot.opts(width=600, height=400)  # Set width and height as needed




:Scatter   [Lng]   (Lat,Humidity,City)

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
city_data_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [3]:
# Narrow down cities that fit criteria using loc
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,wailua homesteads,22.0669,-159.3780,23.44,79,75,2.57,US,1706663124
14,14,adamstown,-25.0660,-130.1015,24.89,76,100,12.26,PN,1706663124
22,22,parham,17.0968,-61.7705,23.35,73,20,1.03,AG,1706663125
23,23,comodoro rivadavia,-45.8667,-67.5000,24.92,29,0,12.35,AR,1706663125
27,27,lihue,21.9789,-159.3672,23.77,79,75,2.57,US,1706663126
...,...,...,...,...,...,...,...,...,...,...
521,521,plettenberg bay,-34.0527,23.3716,21.54,77,88,1.93,ZA,1706663194
529,529,kintinku,-5.8833,35.2333,20.08,96,100,1.57,TZ,1706663195
537,537,tiebo,14.6333,-16.2333,22.06,40,0,1.03,SN,1706663196
539,539,farafangana,-22.8167,47.8333,23.17,85,100,3.47,MG,1706663196


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,fortuna,US,40.5982,-124.1573,56,
1,bredasdorp,ZA,-34.5322,20.0403,95,
2,papatowai,NZ,-46.5619,169.4708,58,
3,udachny,RU,66.4167,112.4000,100,
4,saint-philippe,RE,-21.3585,55.7679,84,
...,...,...,...,...,...,...
542,saint-joseph,RE,-21.3667,55.6167,83,
543,kayangel,PW,8.0823,134.7173,83,
544,yorosso,ML,12.3536,-4.7761,16,
545,serdar,TM,38.9764,56.2757,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": "614e57f3158245beafef67158bc3d79c",
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    print(response)
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
<Response [200]>
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
<Response [200]>
bredasdorp - nearest hotel: Victoria Hotel
<Response [200]>
papatowai - nearest hotel: No hotel found
<Response [200]>
udachny - nearest hotel: Вилюй
<Response [200]>
saint-philippe - nearest hotel: Le Baril
<Response [200]>
edinburgh of the seven seas - nearest hotel: No hotel found
<Response [200]>
nogliki - nearest hotel: Ноглики
<Response [200]>
invercargill - nearest hotel: The Grand
<Response [200]>
grytviken - nearest hotel: No hotel found
<Response [200]>
qianzhou - nearest hotel: JinHuang Hotel
<Response [200]>
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
<Response [200]>
port-aux-francais - nearest hotel: Keravel
<Response [200]>
kodiak - nearest hotel: Shelikof Lodge
<Response [200]>
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
<Response [200]>
adamstown - nearest hotel: No hotel found
<Response [200]>
homer - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
0,fortuna,US,40.5982,-124.1573,56,Comfort Inn & Suites Redwood Country
1,bredasdorp,ZA,-34.5322,20.0403,95,Victoria Hotel
2,papatowai,NZ,-46.5619,169.4708,58,No hotel found
3,udachny,RU,66.4167,112.4000,100,Вилюй
4,saint-philippe,RE,-21.3585,55.7679,84,Le Baril
...,...,...,...,...,...,...
542,saint-joseph,RE,-21.3667,55.6167,83,PALM Hotel & Spa
543,kayangel,PW,8.0823,134.7173,83,No hotel found
544,yorosso,ML,12.3536,-4.7761,16,No hotel found
545,serdar,TM,38.9764,56.2757,74,Chynar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [6]:
import plotly.express as px

# Assuming you have a DataFrame called 'city_data_df' containing city coordinates, hotel names, and countries
# city_data_df should have columns: 'City', 'Lat', 'Lng', 'hotel_name', 'country'

# Configure the map plot
fig = px.scatter_geo(hotel_df, lat='Lat', lon='Lng', hover_name='City',
                     hover_data={'City': True, 'Hotel Name': True, 'Country': True},
                     projection="natural earth")

# Update hover message to include hotel name and country
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>Hotel: %{customdata[0]}<br>Country: %{customdata[1]}')

# Display the map
fig.show()


In [10]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
import pandas as pd



# Create ColumnDataSource
source = ColumnDataSource(hotel_df)

# Create Bokeh plot
p = figure(title="City Map", x_axis_label="Longitude", y_axis_label="Latitude")

# Add scatter points
p.circle(x='Lng', y='Lat', size=10, fill_color='blue', source=source)

# Add hover tool
hover = HoverTool()
hover.tooltips = [
    ("City", "@City"),
    ("Hotel Name", "@{Hotel Name}"),
    ("Country", "@Country")
]
p.add_tools(hover)

# Show plot
show(p)


In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)